In [27]:
DF_PATH = 'c:/Users/Ahmed Sadek/ete_project/ml_project/data/processed/cleaned_data.pkl'
ROLES_COL =['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

MLFLOW_TRACKING_URI = '../ml_project/models/mlruns/'
MLFLOW_EXPERIMENT_NAME ='ete_project'
LOG_PATH = "c:/Users/Ahmed Sadek/ete_project/ml_project/models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"


In [2]:
pip install mlflow 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt

In [4]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

In [5]:
#creating Directories
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

In [7]:
#Read Data
df = pd.read_pickle(DF_PATH)
df

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
3                       0                        0   
4                       0                        0   
8                       0                        0   
9                       0                        0   
...                   ...                      ...   
83434                   0                        0   
83435                   0                        0   
83436                   0                        0   
83437                   0                        0   
83438                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
3                                                 0                      0   
4                                                 0                      0   
8                                                 0                      0   
9                                                 1                      0   
...                                             ...                    ...   
83434                                             0                      0   
83435                                             0                      0   
83436                                             1                      0   
83437                                             0                      0   
83438                                             0                      0   

                                                                   \
      DevOps specialist Developer, QA or test Developer, back-end   
0                     0                     0                   0   
3                     0                     0                   0   
4                     0                     0                   0   
8                     0                     0                   0   
9                     0                     0                   0   
...                 ...                   ...                 ...   
83434                 0                     0                   1   
83435                 0                     0                   0   
83436                 1                     0                   0   
83437                 0                     0                   1   
83438                 0                     0                   1   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 0   
3                                                 0   
4                                                 1   
8                                                 0   
9                                                 0   
...                                             ...   
83434                                             0   
83435                                             0   
83436                                             0   
83437                                             0   
83438                                             0   

                                                                        ...  \
      Developer, embedded applications or devices Developer, front-end  ...   
0                                               0                    0  ...   
3                                               0                    1  ...   
4                                               0                    0  ...   
8                                               0                    1  ...   
9                                               0                    0  ...   
...                                           ...                  ...  ...   
83434                                           0     

In [8]:
#check the total samples of the roles
roles_df = df['DevType'].copy()
roles_df.sum(axis=0)

Academic researcher                               1708
Data or business analyst                          1658
Data scientist or machine learning specialist     2460
Database administrator                            1210
DevOps specialist                                 3056
Developer, QA or test                             1135
Developer, back-end                              17084
Developer, desktop or enterprise applications     4845
Developer, embedded applications or devices       2138
Developer, front-end                              8932
Developer, full-stack                            20655
Developer, game or graphics                        899
Developer, mobile                                 4751
Engineer, data                                    1941
Scientist                                         1046
System administrator                              2069
dtype: int64

In [9]:
#Resample the roles 
samples_per_class = 1200
resampled_role = []
for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col]==1].copy()
    if len(sub_df)< samples_per_class:
        #unsample
        sub_df = sub_df.sample(samples_per_class,replace = True, random_state =0)
    else:
        sub_df = sub_df.sample(samples_per_class, random_state = 0)
    
    resampled_role.append(sub_df)

In [10]:
#constructed df 
roles_df = pd.concat(resampled_role)
df = df.loc[roles_df.index].copy()

In [11]:
roles_df.sum(axis=0)

Academic researcher                              2280
Data or business analyst                         1965
Data scientist or machine learning specialist    2576
Database administrator                           1765
DevOps specialist                                2170
Developer, QA or test                            1514
Developer, back-end                              5710
Developer, desktop or enterprise applications    2690
Developer, embedded applications or devices      1773
Developer, front-end                             2614
Developer, full-stack                            5602
Developer, game or graphics                      1441
Developer, mobile                                2155
Engineer, data                                   2046
Scientist                                        1910
System administrator                             2110
dtype: int64

In [17]:
#splitting
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis= 1),df['DevType'], random_state=0)

C:\Users\Ahmed Sadek\AppData\Local\Temp\ipykernel_10024\3340505829.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis= 1),df['DevType'], random_state=0)


In [28]:
#Trainign Model 
#Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

In [18]:
#logistic regression 
clf= make_pipeline(StandardScaler(),
                  MultiOutputClassifier(LogisticRegression()))
clf.fit(X_train.values,Y_train.values)
predictions= pd.DataFrame(clf.predict(X_train.values), columns= Y_train.columns)

In [19]:
#evaluate on training set
predictions = pd.DataFrame(clf.predict(X_train.values), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train,predictions,score)
               for score in [accuracy_score,precision_score,recall_score,f1_score]}
train_scores = pd.concat(train_scores, axis=1)

In [20]:
#evaluate on testing set 
prediction = pd.DataFrame(clf.predict(X_test.values),columns=Y_test.columns)
test_scores = {score.__name__:calculate_quality(Y_test,prediction,score)
              for score in [accuracy_score,precision_score,recall_score,f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores= test_scores.mean()

In [21]:
print(mean_test_scores)
test_scores

accuracy_score     88.939375
precision_score    62.420625
recall_score       29.890000
f1_score           38.613125
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,90.02,64.42,30.94,41.80
Data or business analyst,91.12,65.99,26.58,37.90
Data scientist or machine learning specialist,89.96,67.58,49.23,56.96
Database administrator,90.90,56.10,5.20,9.52
DevOps specialist,90.21,65.38,25.47,36.66
"Developer, QA or test",91.71,28.57,0.51,1.00
"Developer, back-end",74.40,62.64,35.72,45.50
"Developer, desktop or enterprise applications",87.52,61.58,18.20,28.09
"Developer, embedded applications or devices",92.65,64.79,33.17,43.88
"Developer, front-end",88.40,67.91,36.76,47.70


In [23]:
train_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,89.24,61.32,27.49,37.97
Data or business analyst,90.68,62.84,22.22,32.83
Data scientist or machine learning specialist,90.59,71.08,50.10,58.78
Database administrator,90.94,56.16,6.20,11.16
DevOps specialist,90.44,67.06,31.23,42.62
"Developer, QA or test",92.24,56.25,0.80,1.59
"Developer, back-end",74.31,61.75,35.28,44.91
"Developer, desktop or enterprise applications",86.88,62.98,18.61,28.73
"Developer, embedded applications or devices",92.17,66.24,34.41,45.30
"Developer, front-end",88.97,65.81,36.14,46.66


In [22]:
#Data_details 
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [24]:
#model
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(clf),
         "model_object": clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [25]:
# Performance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [29]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 

AttributeError: 'NoneType' object has no attribute 'experiment_id'